## Imports and settings

In [1]:
import src
import keras.backend as K
import os
import numpy as np
import sys
import re
import math
import io
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from  matplotlib.animation import FuncAnimation
from matplotlib import colors
from netCDF4 import Dataset
from IPython.display import clear_output
#data folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/data')
#forces CPU usage
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #"" or "-1" for CPU, "0" for GPU
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11149758456406783193
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1508248780
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5903658833352000161
physical_device_desc: "device: 0, name: GeForce GT 740M, pci bus id: 0000:01:00.0, compute capability: 3.5"
]


### Advection layer

In [2]:
def advect(image): # (64,64,3)
    """
    Applies the physical advection (material derivative) on a density (rain) frame.
    See: https://en.wikipedia.org/wiki/Advection
    in short: r(t+1)=r(t)-vx(t)*drdx(t)-vy(t)*drdy(t)
    :param image: one image with 3 channels: the advected material (rain desity) and the flow field (wind) x and y components. 
    """
    #pad image
    padded = np.pad(image,(0,1),'edge')[:,:,:-1]
    #set nans to 0
    padded[np.isnan(padded)] = 0
    #create array for advected frame
    advected = np.empty_like(image)
    #advect (nans will be treated as 0s)
    advected[:,:,0] = image[:,:,0] - image[:,:,1]*(padded[1:,:,0] - padded[:-1,:,0])[:,:-1] - image[:,:,2]*(padded[:,1:,0] - padded[:,:-1,0])[:-1]
    #renormalize (saturate)
    advected[:,:,0][advected[:,:,0] < 0] = 0
    advected[:,:,0][advected[:,:,0] > 1] = 1   
    #other channels stay the same
    advected[:,:,1:] = image[:,:,1:]
    return advected[:,:,0:1]  # (64, 64, 1) only rain

### GAN

In [7]:
#modified from source: https://github.com/eriklindernoren/Keras-GAN/blob/master/gan/gan.py
class GAN():
    def __init__(self, dual=False, past=1, loss_function="mae", augment=False):
        self.dual = dual #set this to True to train temporal discriminator
        self.size = 64
        self.past_input = past #set this to change sequence length
        self.tempoGAN_sequence_length = past
        self.input_shape = (self.size, self.size, self.past_input) # 64, 64, t
        self.d_metric = ["accuracy"]
        self.log = {"g_loss":[],
               "d_loss":[],
               "g_metric":[],
               "d_metric":[]}
        self.inputs = []
        self.outputs = []
        self.losses = [loss_function, "binary_crossentropy"]
        self.train_data = None
        self.xval_data = None
        self.test_data = None
        self.augment = augment
        
        d_optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        g_optimizer = keras.optimizers.Adam(0.0002, 0.5)
        
# Build the generator
        self.generator = self.build_generator()
        # The generator takes a sequence of frames as input and generates the next image in that sequence
        input_img = keras.layers.Input(shape=self.input_shape)
        self.inputs.append(input_img)
        generated = self.generator(input_img)
        self.outputs.append(generated)
        
# Build and compile spatial discriminator
        self.s_discriminator = self.build_discriminator("s")
        self.s_discriminator.compile(loss='binary_crossentropy',
            optimizer=d_optimizer,
            metrics=self.d_metric)
        # Spatial disc. takes the x as condition and G(x) and returns a float
        score_s = self.s_discriminator([input_img, generated])
        self.outputs.append(score_s)
        self.s_discriminator.trainable = False
        
# Build and compile temporal discriminator (same as s disc. but has different inputs
        if self.dual:
            self.t_discriminator = self.build_discriminator("t")
            self.t_discriminator.compile(loss='binary_crossentropy',
                optimizer=d_optimizer,
                metrics=self.d_metric)
            #Temporal disc. takes in advected frame A(G(x_previous)) and G(x)
            adv = keras.layers.Input(shape=self.input_shape)
            self.inputs.append(adv)
            score_t = self.t_discriminator([adv, generated])
            self.outputs.append(score_t)
            self.t_discriminator.trainable = False  
            self.losses.append('binary_crossentropy')
        
#Combined GAN model
        self.combined = keras.models.Model(inputs=self.inputs, outputs=self.outputs)
        #loss on all ouputs as a list: l1 loss on generated img, cross entropy for discriminator
        self.combined.compile(loss=self.losses, optimizer=g_optimizer)

    def build_generator(self,network="U-net"):  
        generator = keras.Sequential()
        if network in ["Unet", "U-net", "unet", "u-net"]:
            return src.unet(self.input_shape)  # 64, 64, t

    def build_discriminator(self, which="s"):
        if which == "s":
            return src.spatial_discriminator(condition_shape=self.input_shape)
        elif which == "t":
            return src.temporal_discriminator()
# ---------------------
#  Train
# ---------------------
    def train(self, epochs, d_epochs=1, dataset="5min", batch_size=128):
        assert isinstance(d_epochs, int) > 0 and isinstance(epochs, int) > 0 , "Number of epochs must be a positive integer."
        
# Load the dataset
        if self.dual:
            if dataset not in ["gan", "GAN", "tempogan", "tempoGAN"]:
                dataset = "gan"
                print("tempoGAN training: Changed dataset to GAN data.")
            self.past_input += 1
            print("tempoGAN training: Increased input sequence length by one. First frame is only auxiliary for advection.")
        else:
            if dataset in ["gan", "GAN", "tempogan", "tempoGAN"]:
                dataset = "5min"
                print("Normal GAN training: Changed dataset to 5min data.")

            
        print(f"Loading {dataset} dataset.")
        self.train_data, self.xval_data, self.test_data = src.load_datasets(dataset, self.past_input)
        #replace nans with -1 (can cause bias for velocity fields)
        #self.train_data[np.isnan(self.train_data)] = -1
        #self.xval_data[np.isnan(self.xval_data)] = -1
        #self.test_data[np.isnan(self.test_data)] = -1
        
# split the dataset to inputs and ground truths
        gan_train, gan_truth, gan_val, gan_val_truth, gan_test, gan_test_truth = src.split_datasets(
            self.train_data[:2000], self.xval_data, self.test_data, past_frames=self.past_input, augment=self.augment)
        
# Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        for epoch in range(epochs):

# ---------------------
#  Train Discriminators
# ---------------------

# Train the first discriminator
#inputs: [frame t, generated frame t+1 (from frame t)] & [frame t, ground truth of frame t (frame t+1)]
#batches are unmixed
            self.s_discriminator.trainable = True
            for ks in range(d_epochs):
                # all 4D
                real_imgs, training_batch, generated_imgs, _, _ = self.create_training_batch(gan_train, gan_truth, batch_size)
                ds_loss_real = self.s_discriminator.train_on_batch([training_batch, real_imgs], real)
                ds_loss_fake = self.s_discriminator.train_on_batch([training_batch, generated_imgs], fake)
                ds_loss = 0.5 * np.add(ds_loss_real, ds_loss_fake)
                if d_epochs > 1:
                    print(f"    {ks} [Ds loss: {ds_loss[0]}, acc.: {100*ds_loss[1]}]")
            d_loss = ds_loss
            self.s_discriminator.trainable = False
                
# Train the second discriminator
#inputs: [advected generated frame t (from frame t-1), generated frame t+1 (from frame t)] &
#        [advected ground truth of frame t-1 (advected frame t), ground truth frame t (frame t+1)]
#batches are unmixed
            if self.dual:
                self.t_discriminator.trainable = True
                for kt in range(d_epochs):
                    # 4D, 4D, 4D, 4D, 4D
                    real_imgs, training_batch, generated_imgs, advected_aux_gen, advected_aux_truth = self.create_training_batch(
                                                                                        gan_train, gan_truth, batch_size)
                    #only need rain map from the synthetics
                    dt_loss_real = self.t_discriminator.train_on_batch([advected_aux_truth, real_imgs], real)
                    dt_loss_fake = self.t_discriminator.train_on_batch([advected_aux_gen, generated_imgs], fake)
                    dt_loss = 0.5 * np.add(dt_loss_real, dt_loss_fake)
                    if d_epochs > 1:
                        print(f"    {kt} [Dt loss: {dt_loss[0]}, acc.: {100*dt_loss[1]}]")
                d_loss = ds_loss + dt_loss
                self.t_discriminator.trainable = False
            
# ---------------------
#  Train Generator
# ---------------------

            idx = np.random.randint(0, gan_train.shape[0], batch_size)
            if self.dual:
                training_truth = gan_truth[idx,:,:,:,0]  # frame t+1, 4D: n, 64, 64, 1
                assert training_truth.shape[-1] == 1, f"real_imgs: (n, 64, 64, 1), {real_imgs.shape}"
                aux_batch = gan_train[idx,:,:,:-1, 0]  # from 0 to frame t-1 (not the last frame) 4D: n, 64, 64, past-1
                assert aux_batch.shape[-1] == self.past_input-1, f"aux_batch: (n, 64, 64, {self.past_input-1}), {aux_batch.shape}"
                training_batch = gan_train[idx,:,:,1:,0]  # from frame 1 to end (t>=2), 4D: n, 64, 64, past-1
                assert training_batch.shape[-1] == self.past_input-1, f"training_batch: (n, 64, 64, {self.past_input-1}), {training_batch.shape}"
                aux_gen_imgs = self.generator.predict(aux_batch) # 4D, n, 64, 64, 1: output is frame t  
                assert aux_gen_imgs.shape[-1] == 1, f"aux_gen_imgs: (n, 64, 64, 1), {aux_gen_imgs.shape}"
                # append velocity field of frame t (last instance of past sequence)
                aux_gen_imgs = np.concatenate((aux_gen_imgs, gan_train[idx,:,:,-1,1:]), axis=-1) #n, 64, 64, 3
                assert aux_gen_imgs.shape[-1] == 3, f"aux_gen_imgs: (n, 64, 64, 3), {aux_gen_imgs.shape}"
                # advect generated frame t
                advected_aux_gen = np.array([advect(sample) for sample in aux_gen_imgs]) #4D (n, h, w, m) (m: rho, vx, vy)
                assert advected_aux_gen.shape[-1] == 1, f"advected_aux_gen: (n, 64, 64, 1), {advected_aux_gen.shape}"
            else:
                training_batch = gan_train[idx]  # frame t or all past frames, 4D
                training_truth = gan_truth[idx]  # frame t+1 or all future frames, 4D

# Train the generator (to have the discriminator label samples as real)
            if self.dual:
                g_loss = self.combined.train_on_batch([training_batch, advected_aux_gen], [training_truth, real, real])
            else:
                g_loss = self.combined.train_on_batch(training_batch, [training_truth, real])

# Plot the progress
            self.log["g_loss"].append(g_loss)
            self.log["d_loss"].append(d_loss[0])
            #log["g_metric"].append(g_loss[1])
            self.log["d_metric"].append(d_loss[1])
            print(f"\033[1m {epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}]\033[0m"+
                  f"\033[1m[G loss: {g_loss}]\033[0m")#, rel. err.: {g_loss[1]}] \033[0m")

# If at save interval => save generated image samples
            if epoch in [int(x) for x in np.linspace(0.01,5,100)*epochs]:
                self.sample_images(epoch, gan_test, gan_test_truth)
    
    def create_training_batch(self, gan_train, gan_truth, batch_size):
        idx = np.random.randint(0, gan_truth.shape[0], batch_size)
        # Generate a batch of new images
        if self.dual:
            #0,1->2
            real_imgs = gan_truth[idx,:,:,:,0]  # frame t+1, 4D: n, 64, 64, 1
            assert real_imgs.shape[-1] == 1, f"real_imgs: (n, 64, 64, 1), {real_imgs.shape}"
            training_batch = gan_train[idx,:,:,1:,0]  # from frame 1 to end (t>=2), 4D: n, 64, 64, past-1
            assert training_batch.shape[-1] == self.past_input-1, f"training_batch: (n, 64, 64, {self.past_input-1}), {training_batch.shape}"
            aux_batch = gan_train[idx,:,:,:-1, 0]  # from 0 to frame t-1 (not the last frame) 4D: n, 64, 64, past-1
            assert aux_batch.shape[-1] == self.past_input-1, f"aux_batch: (n, 64, 64, {self.past_input-1}), {aux_batch.shape}"
            generated_imgs = self.generator.predict(training_batch) # n, h, w, 1, rho (4 dimensional, last drops)
            assert generated_imgs.shape[-1] == 1, f"generated_imgs: (n, 64, 64, 1), {generated_imgs.shape}"
            aux_gen_imgs = self.generator.predict(aux_batch) # 4D, n, 64, 64, 1: output is frame t 
            assert aux_gen_imgs.shape[-1] == 1, f"aux_gen_imgs: (n, 64, 64, 1), {aux_gen_imgs.shape}"
            # append velocity fields of frame t
            #this will be advected
            aux_gen_imgs = np.concatenate((aux_gen_imgs, gan_train[idx,:,:,-1,1:]), axis=-1) #n, 64, 64, 3
            assert aux_gen_imgs.shape[-1] == 3, f"aux_gen_imgs: (n, 64, 64, 3), {aux_gen_imgs.shape}"
            aux_true_imgs = gan_train[idx,:,:,-1] #n, 64, 64, 3, frame t with all channels
            assert aux_true_imgs.shape[-1] == 3, f"aux_true_imgs: (n, 64, 64, 3), {aux_true_imgs.shape}"
            # advected frame t (frame t+1)
            advected_aux_gen = np.array([advect(sample) for sample in aux_gen_imgs]) #4D (n, h, w, m) (m: rho, vx, vy)
            assert advected_aux_gen.shape[-1] == 1, f"advected_aux_gen: (n, 64, 64, 1), {advected_aux_gen.shape}"
            advected_aux_truth = np.array([advect(sample) for sample in aux_true_imgs]) #4D
            assert advected_aux_truth.shape[-1] == 1, f"advected_aux_truth: (n, 64, 64, 1), {advected_aux_truth.shape}"
            
        else: # 4D
            real_imgs = gan_truth[idx] # 4D
            training_batch = gan_train[idx] # 4D
            generated_imgs = self.generator.predict(training_batch) #4D
            advected_aux_gen = None
            advected_aux_truth = None
        return real_imgs, training_batch, generated_imgs, advected_aux_gen, advected_aux_truth # all 4D
    
    def sample_images(self, epoch, gan_test, gan_test_truth):
        n = 5
        if self.dual:
            test_batch = gan_test[:n,:,:,1:,0]  # frame 1 to t (0 is not used bc. its only used in advection), 4D
            test_truth = gan_test_truth[:n,:,:,:,0] #4th dim is always 1 so ":" is OK
        else:
            test_batch = gan_test[:n]
            test_truth = gan_test_truth[:n]
        gen_imgs = self.generator.predict(test_batch)

        fig, axs = plt.subplots(n, 3, figsize=(16, 16))
        for i in range(n):
                axs[i,0].imshow(test_batch[i, :,:,0])
                axs[i,0].axis('off')
                axs[i,0].set_title("Frame t")
                axs[i,1].imshow(test_truth[i, :,:,0])
                axs[i,1].axis('off')
                axs[i,1].set_title("Frame t+1")
                axs[i,2].imshow(gen_imgs[i, :,:,0])
                axs[i,2].axis('off')
                axs[i,2].set_title("Prediction t+1")
        fig.savefig("Plots/epoch %d.png" % epoch)
        plt.close()

In [9]:
gan= GAN(dual=False)
gan.combined.loss

['mae', 'binary_crossentropy']

In [10]:
gan.train(epochs=10,dataset="gan", d_epochs=1, batch_size=64)

Normal GAN training: Changed dataset to 5min data.
Loading 5min dataset.
Training data: (7500, 64, 64, 2)
Validation data: (1500, 64, 64, 2)
Test data: (1000, 64, 64, 2)
Shape of training data:  (2000, 64, 64, 1) 
Shape of training truth:  (2000, 64, 64, 1) 
Shape of validation data:  (1500, 64, 64, 1) 
Shape of validation truth:  (1500, 64, 64, 1) 
Shape of test data:  (1000, 64, 64, 1) 
Shape of test truth:  (1000, 64, 64, 1)
 0 [D loss: 0.7011295557022095, acc.: 40.625][G loss: [0.84237826, 0.1474333, 0.694945]]
 1 [D loss: 0.7029887437820435, acc.: 35.9375][G loss: [0.85200715, 0.16640535, 0.6856018]]
 2 [D loss: 0.705133318901062, acc.: 32.8125][G loss: [0.8560422, 0.17331588, 0.6827263]]
 3 [D loss: 0.7005679607391357, acc.: 42.1875][G loss: [0.84826064, 0.16814318, 0.6801175]]
 4 [D loss: 0.6948080062866211, acc.: 44.53125][G loss: [0.8606358, 0.18312973, 0.6775061]]
 5 [D loss: 0.6962077617645264, acc.: 35.15625][G loss: [0.8256122, 0.14494458, 0.68066764]]
 6 [D loss: 0.694727

In [ ]:
f = plt.figure(figsize=(16,4))
ax = f.add_subplot(121)
ax2 = f.add_subplot(122)

g_labels = ["Generator loss"]+gan.combined.loss
#loop over gen loss components
for curve in range(np.shape(gan.log["g_loss"])[-1]):
    ax.plot(np.array(gan.log["g_loss"])[:,curve] ,label=g_labels[curve])
ax.plot(gan.log["d_loss"],label="Discriminator loss")
ax.grid()
ax.set_xlabel("Iterations")
ax.set_ylabel("Loss")
ax.legend(loc="best")

ax2.plot(gan.log["g_metric"],label="Generator metric")
ax2.plot(gan.log["d_metric"],label="Discriminator metric")
ax2.grid()
ax2.set_xlabel("Iterations")
ax2.set_ylabel("Accuracy")
ax2.legend(loc="best")

plt.show()

In [ ]:
np.shape(gan.log["g_loss"])[-1]

______________________________________-
## Load datasets

__<font color='red'>SHOW</font>__

In [ ]:
train, xval, test = src.load_datasets("5min")

In [ ]:
split_datasets(train, xval, test, 4, 4)

In [ ]:
def split_datasets(train, xval, test, past_frames=1, future_frames=1):
    """
    Further splits data to input and ground truth datasets.
    :param train: numpy array of training dataset
    :param xval: numpy array of validation dataset
    :param test: numpy array of test dataset
    :param past_frames: int, no. of consec. frames that will be the input of the network
    :param future_frames: int, no. of consec. frames that will be the ground truth of the network
    :return: 6 numpy arrays
    """
    assert past_frames + future_frames <= train.shape[1], "Wrong frame specification!"
    assert past_frames > 0, "No. of past frames must be a positive integer!"
    assert future_frames > 0, "No. of future frames must be a positive integer!"
    training_data = train[:, :, :, :past_frames]
    trainig_data_truth = train[:, :, :, past_frames:past_frames + future_frames]
    validation_data = xval[:, :, :, :past_frames]
    validation_data_truth = xval[:, :, :, past_frames:past_frames + future_frames]
    test_data = test[:, :, :, :past_frames]
    test_data_truth = test[:, :, :, past_frames:past_frames + future_frames]

    print("Shape of training data: ", training_data.shape, "\nShape of training truth: ", trainig_data_truth.shape,
          "\nShape of validation data: ", validation_data.shape, "\nShape of validation truth: ",
          validation_data_truth.shape,
          "\nShape of test data: ", test_data.shape, "\nShape of test truth: ", test_data_truth.shape)
    return training_data, trainig_data_truth, validation_data, validation_data_truth, test_data, test_data_truth

In [ ]:
#train[np.isnan(train)] = -2
#xval[np.isnan(xval)] = -2
#test[np.isnan(test)] = -2
# split the dataset
past_frames = 1 # at least 2 frames needed into the temporal discriminator
future_frames = 1
gan_train, gan_truth, gan_val, gan_val_truth, gan_test, gan_test_truth2 = src.split_datasets(
    train, xval, test, past_frames, future_frames)

In [ ]:
indices = np.random.randint(0,len(train),10)

for i in indices:
    data = [train[i,0,:,:,0], train[i,1,:,:,0], train[i,2,:,:,0],
            train[i,2,:,:,1], train[i,2,:,:,2], train[i,3,:,:,0]]
    fig, axes = plt.subplots(nrows=1, ncols=6, num=None, figsize=(16, 16), dpi=80, facecolor='w', edgecolor='k')
    for j, ax in enumerate(axes.flat):
        im = ax.imshow(data[j],cmap="seismic" if j in [3,4] else None, vmin=0 if j in [0,1,2,5] else -1,
                          vmax=max([np.max(dset[j]) if j in [0,1,2,5] else 1 for dset in data]) )
           # , norm=colors.PowerNorm(gamma=0.5) if int(j) == 3 else None)
        ax.set_title(f"Index: {i}, Frame: {j}", fontsize=10)
        src.colorbar(im)
        ax.axis('off')

In [ ]:
reloaded = load_model("spatial_GAN_5000.h5")
log = np.load("log_Spatial_GAN_5000.npy").item()

In [ ]:
generator = reloaded.layers[1]
discriminator = reloaded.layers[2]

In [ ]:
labels = discriminator.predict([predictions,gan_test])

In [ ]:
labels

In [ ]:
predictions = generator.predict(gan_test)

In [ ]:
args = src.arg_getter(gan_test_truth2, predictions)
args[-1]

In [ ]:
error_images, error_vals, error_means = src.error_distribution(gan_test_truth2,predictions, metric="difference")

In [ ]:
%matplotlib inline
src.result_plotter(args[-5:], (gan_test2[:,:,:,0], gan_test_truth2[:,:,:,0], predictions[:,:,:,0], error_images[:,:,:,0]))

In [ ]:
#GAN learns:
#sharper but not more accurate
#small rain patches disappear
#learns an average wind motion: mostly to the right: applies that everywhere--> data augmentation based on optical flow
#hourly dataset with wind is still not accurate enough

In [ ]:
#train unet for 5min data, 1-1, 2-1, 3-1, best-more
#train a single disc gan for the 5min and the h data, 1-1, 2-1, 3-1, best-more
#train dual disc for hour data, 1-1, 2-1, 3-1, best-more

In [ ]:
#4 frames to predict one future frame, use that as input for further predictions

In [ ]:
test =gan_truth

In [ ]:
test.shape

In [ ]:
plt.subplot(4,4,2)
plt.imshow(augmented[0,:,:,0])
plt.axis("off")
plt.subplot(4,4,3)
plt.imshow(augmented[4,:,:,0])
plt.axis("off")
plt.subplot(4,4,8)
plt.imshow(augmented[1,:,:,0])
plt.axis("off")
plt.subplot(4,4,12)
plt.imshow(augmented[5,:,:,0])
plt.axis("off")
plt.subplot(4,4,14)
plt.imshow(augmented[2,:,:,0])
plt.axis("off")
plt.subplot(4,4,15)
plt.imshow(augmented[6,:,:,0])
plt.axis("off")
plt.subplot(4,4,5)
plt.imshow(augmented[3,:,:,0])
plt.axis("off")
plt.subplot(4,4,9)
plt.imshow(augmented[7,:,:,0])
plt.axis("off")
plt.savefig("augmentation")

In [ ]:
augmented = augment_data(test)

In [ ]:
augmented.shape

In [ ]:
def augment_data(data):
    #dimensions are n, h, w, c
    print("Data augmentig done.")
    return np.reshape([np.array([
        data_sample,
        rotate(data_sample, "90"),
        rotate(data_sample, "180"),
        rotate(data_sample, "270"),
        np.flip(data_sample, axis=1),
        np.flip(rotate(data_sample, "90"), axis=1),
        np.flip(rotate(data_sample, "180"), axis=1),
        np.flip(rotate(data_sample, "270"), axis=1)]) for data_sample in data], ((data.shape[0]*8,)+data.shape[1:]))

def rotate(img, degree):

    assert degree in ["90","-270", "180", "-90", "270"], "Rotation degree must be in: [90, 180, 270, -90, -270]"
    rotated = np.rot90(img)
    if degree in ["180", "-90", "270"]:
        rotated = np.rot90(rotated)
    if degree in ["-90", "270"]:
        rotated = np.rot90(rotated)
    return rotated